In [3]:
# !pip install --upgrade torch torchvision
# !pip install easyfsl

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

import os
import shutil
from sklearn.model_selection import train_test_split


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 72.8/72.8 kB 401.0 kB/s eta 0:00:00
     ---------------------------------------- 11.6/11.6 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 346.6/346.6 kB 1.8 MB/s eta 0:00:00
  Using cached torch-2.4.1-cp39-cp39-win_amd64.whl (199.3 MB)
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2022.6
    Uninstalling tzdata-2022.6:
      Successfully uninstalled tzdata-2022.6
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 203.0/203.0 MB 891.3 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.1
    Uninstalling torchvision-0.19.1:
      Successfully uninstalled torchvision-0.19.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.1+cpu requires torch==2.4.1, but you have torch 2.5.1 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable


AttributeError: module 'torch' has no attribute 'Tensor'

In [ ]:

# Paths
base_path = r"C:\Users\hp\Downloads\neem images_realtime_HealthyDiseased\neem images_realtime_HealthyDiseased"
healthy_path = os.path.join(base_path, "healthy")
diseased_path = os.path.join(base_path, "diseased")
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

# Create train/test directories
for category in ["healthy", "diseased"]:
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(test_path, category), exist_ok=True)

# Split data
def split_and_move_images(source_path, train_dest, test_dest, test_size=0.2):
    images = [f for f in os.listdir(source_path) if os.path.isfile(os.path.join(source_path, f))]
    train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)

    # Move images to train and test folders
    for image in train_images:
        shutil.move(os.path.join(source_path, image), os.path.join(train_dest, image))
    for image in test_images:
        shutil.move(os.path.join(source_path, image), os.path.join(test_dest, image))

# Perform splitting for healthy and diseased images
split_and_move_images(healthy_path, os.path.join(train_path, "healthy"), os.path.join(test_path, "healthy"))
split_and_move_images(diseased_path, os.path.join(train_path, "diseased"), os.path.join(test_path, "diseased"))

print("Dataset split completed.")




In [ ]:
# Set parameters
image_size = 224
N_WAY = 2
N_SHOT = 5
N_QUERY = 10
N_TRAINING_EPISODES = 40000
N_VALIDATION_TASKS = 100
N_EVALUATION_TASKS = 100
num_epochs = 10

# Load datasets with enhanced transformations
transform_train = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_set = ImageFolder(root=train_path, transform=transform_train)
test_set = ImageFolder(root=test_path, transform=transform_test)



In [ ]:
# Prototypical Networks Model
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone

    def forward(self, support_images, support_labels, query_images):
        z_support = self.backbone(support_images)
        z_query = self.backbone(query_images)
        n_way = len(torch.unique(support_labels))
        z_proto = torch.cat([
            z_support[torch.nonzero(support_labels == label)].mean(0)
            for label in range(n_way)
        ])
        dists = torch.cdist(z_query, z_proto)
        return -dists

# Initialize the model
convolutional_network = resnet18(pretrained=True)
convolutional_network.fc = nn.Identity()  # Remove FC layer for feature extraction

# Fine-tune the last ResNet layers
for param in convolutional_network.parameters():
    param.requires_grad = False
for param in convolutional_network.layer4.parameters():
    param.requires_grad = True

model = PrototypicalNetworks(convolutional_network).cuda()



In [ ]:
# Data samplers and loaders
train_set.get_labels = lambda: [instance[1] for instance in train_set.samples]
train_sampler = TaskSampler(train_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_TRAINING_EPISODES)
train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=12, pin_memory=True, collate_fn=train_sampler.episodic_collate_fn)

test_set.get_labels = lambda: [instance[1] for instance in test_set.samples]
test_sampler = TaskSampler(test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS)
test_loader = DataLoader(test_set, batch_sampler=test_sampler, num_workers=12, pin_memory=True, collate_fn=test_sampler.episodic_collate_fn)

# Loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.5)



In [ ]:
# Training function
def fit(support_images, support_labels, query_images, query_labels):
    optimizer.zero_grad()
    classification_scores = model(support_images.cuda(), support_labels.cuda(), query_images.cuda())
    loss = criterion(classification_scores, query_labels.cuda())
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
model.train()
all_loss = []
with tqdm(enumerate(train_loader), total=len(train_loader)) as tqdm_train:
    for episode_index, (support_images, support_labels, query_images, query_labels, _) in tqdm_train:
        loss_value = fit(support_images, support_labels, query_images, query_labels)
        all_loss.append(loss_value)
        scheduler.step()

        if episode_index % 10 == 0:
            tqdm_train.set_postfix(loss=sliding_average(all_loss, 10))

# Evaluation function
def evaluate(data_loader):
    total_predictions = 0
    correct_predictions = 0
    model.eval()
    with torch.no_grad():
        for support_images, support_labels, query_images, query_labels, class_ids in tqdm(data_loader):
            classification_scores = model(support_images.cuda(), support_labels.cuda(), query_images.cuda())
            predictions = torch.argmax(classification_scores, dim=1)
            correct_predictions += (predictions == query_labels.cuda()).sum().item()
            total_predictions += len(query_labels)

    accuracy = 100 * correct_predictions / total_predictions
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Evaluate on test set
test_accuracy = evaluate(test_loader)